In [14]:
# MTEB: Massive text embedding model 
# https://huggingface.co/spaces/mteb/leaderboard

# open source llm https://huggingface.co/google/gemma-7b-it
#create write token from huggingface
# gemma-7b-it, gemma-2b-it
import os
# Flag to print debug messages/write debug files
local_debug = True
# Remove huggingface token after execution for security purpose and comment below line
#os.environ['HF_TOKEN'] = '<token>'


In [2]:
from sentence_transformers import SentenceTransformer

def get_query_embedding(input_query: str) -> list:
    embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2')    
    query_embedding = embedding_model.encode(input_query)
    query_embedding_list = query_embedding.tolist()        
    return query_embedding_list

C:\Users\anila\ml\mlenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# CREATE OR REPLACE FUNCTION get_embedding(	
# 	rows_limit int,
#     embedding_input vector(768)
# )
# RETURNS TABLE (id bigint, chunk varchar(2500), embedding vector(768), cosine_similarity DOUBLE PRECISION)
# LANGUAGE plpgsql
# AS $$
# BEGIN    
# 	RETURN QUERY SELECT ni.id,ni.chunk, ni.embedding, 1 - (ni.embedding <=> embedding_input) AS cosine_similarity FROM nutritionitems ni
# 	ORDER BY cosine_similarity DESC LIMIT rows_limit;
# END;
# $$;

# select get_embedding(2::int,'[5.99734336e-02,-1.30569497e-02]'::vector);


In [4]:
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv
from psycopg2 import Error

class PostgreSQLManager:
    def __init__(self, db_params):
        """
        Initializes the database connection.
        db_params should be a dictionary with keys like 'host', 'database', 'user', 'password', 'port'.
        """
        self.db_params = db_params
        self.connection = None

    def connect(self):
        """Establishes a connection to the PostgreSQL database."""
        try:
            self.connection = psycopg2.connect(**self.db_params)
            self.connection.autocommit = False  # Disable autocommit for explicit transactions
            print("Database connection established successfully.")
        except Error as e:
            print(f"Error connecting to database: {e}")
            self.connection = None

    def disconnect(self):
        """Closes the database connection."""
        if self.connection:
            self.connection.close()
            print("Database connection closed.")

    def execute_query(self, query, params=None, fetch_result=False):
        """Helper method to execute a query and handle transactions."""
        if not self.connection:
            print("No database connection. Please connect first.")
            return None

        try:
            with self.connection.cursor() as cursor:
                if params:
                    cursor.execute(query, params)
                else:
                    cursor.execute(query)

                if fetch_result:
                    return cursor.fetchall()
                else:
                    self.connection.commit()  # Commit changes for CUD operations
                    return True
        except Error as e:
            self.connection.rollback()  # Rollback on error
            print(f"Database operation failed: {e}")
            return None

In [5]:
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv

def get_relevant_chunks(limit:int, query_embedding_list:str) -> list:
    load_dotenv()
    DB_NAME = os.getenv("DB_NAME")
    DB_USER = os.getenv("DB_USER")
    DB_PASS = os.getenv("DB_PASS")
    DB_HOST = os.getenv("DB_HOST")
    DB_PORT = os.getenv("DB_PORT")
    db_params = {
            "host": DB_HOST,
            "database": DB_NAME,
            "user": DB_USER,
            "password": DB_PASS,
            "port": DB_PORT
        }

    crud_manager = PostgreSQLManager(db_params)
    crud_manager.connect()

    if crud_manager.connection:
        select_sql = """SELECT * from get_embedding(%s,%s::vector);"""
        nutritionitems = crud_manager.execute_query(select_sql, (limit,query_embedding_list), True)
        print("nutritionitems len", len(nutritionitems))
        retrieved_chunks_dict = []
        retrieved_chunks = []
        for item in nutritionitems:
            retrieved_chunks.append(item[1])
            retrieved_chunks_dict.append({"id":item[0],
                                     "chunk": item[1],
                                     "cosine_similarity": item[3]
                                    })
        crud_manager.disconnect()
        if local_debug:
            retrieved_chunks_file = "Retrieved_chunks_test.txt"        
            if os.path.exists(retrieved_chunks_file):
                os.remove(retrieved_chunks_file)        
            
            with open(retrieved_chunks_file, 'w', encoding='utf-8') as file:
                for item in retrieved_chunks_dict :  
                    file.write(f"{item["id"]} | {item["cosine_similarity"]} | {item["chunk"]} \n\n")
        return retrieved_chunks
        
    

In [6]:
def prompt_formatter(query:str,
                    context_items: list[dict])-> str:
    context = "- "+"\n- ".join([item for item in context_items])            
    
    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query
Don't return the thinking, only return answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and vitamin K. These vitamins dissolve in fat and are absorbed and stored in the body's fatty tissues
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Insulin Resistance – Body’s cells don’t respond properly to insulin, leading to elevated blood glucose. Pancreatic Dysfunction – Over time, the pancreas can’t produce enough insulin to compensate for resistance.
\nNow use the following context items to answer user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""
    base_prompt = base_prompt.format(context=context, query=query)
    return base_prompt

In [7]:
from huggingface_hub import login
import os
def connect_to_huggingface():
    hf_token_val = os.getenv('HF_TOKEN')
    login(token=hf_token_val)


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def chat_template(formatted_prompt: str) -> str:
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    # Define a chat history
    messages = [
        {"role": "user", "content": formatted_prompt}    
    ]    
    # Apply the chat template, which automatically handles BOS/EOS and formatting
    chat_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    prompt_file = "prompt_test.txt"    
    if os.path.exists(prompt_file):
        os.remove(prompt_file)    
    with open(prompt_file, 'w', encoding='utf-8') as file:
        file.write(chat_prompt)
    return chat_prompt

In [9]:
def llm_generate(chat_prompt: str) -> str:
    model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2b-it",
        torch_dtype=torch.bfloat16
    )    
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=250)
    output = tokenizer.decode(outputs[0])
    return output    

In [11]:

# No Answer
#input_query = "Describe the process of digestion and absorption of nutrients in the human body."
#input_query = "What is Metabolism of Proteins?"
input_query = "Explain Digestion and Metabolism of Carbohydrates"
query_embedding_list = get_query_embedding(input_query)
if local_debug :    
    print("query_embedding len", len(query_embedding_list))

retrieved_chunks = get_relevant_chunks(5, query_embedding_list)
if local_debug :
    print("retrieved_chunks len", len(retrieved_chunks))

formatted_prompt = prompt_formatter(input_query,retrieved_chunks)
if local_debug :
    print("formatted_prompt len", len(formatted_prompt))
connect_to_huggingface()

chat_prompt = chat_template(formatted_prompt)
if local_debug :
    print("chat_prompt len", len(chat_prompt))
llm_output = llm_generate(chat_prompt)
print("llm_output: -----", llm_output)    


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


query_embedding len 768
Database connection established successfully.
nutritionitems len 5
Database connection closed.
retrieved_chunks len 5
formatted_prompt len 9865


`torch_dtype` is deprecated! Use `dtype` instead!


chat_prompt len 9925


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 17.46it/s]


NameError: name 'tokenizer' is not defined